# 🧮 Tutorial 2: Geometric Calculations & Route Analysis

## 🎯 What You'll Learn
- Calculate room areas with Shapely
- Build circulation graphs with NetworkX
- Find shortest evacuation paths
- Visualize building connectivity

---


In [ ]:
import json
from shapely.geometry import Polygon
import networkx as nx
import matplotlib.pyplot as plt

# Load project data
with open('data/extracted/I01.4_mock_data_converted.json', 'r') as f:
    project = json.load(f)

print("📊 Calculating room areas...\n")

for room in project['rooms']:
    # Convert boundary to polygon
    polygon = Polygon(room['boundary'])
    
    # Calculate area
    area = polygon.area
    
    print(f"🏢 {room['name']}")
    print(f"   Area: {area:.2f} m²")
    print(f"   Perimeter: {polygon.length:.2f} m")
    print()


In [ ]:
# Create circulation graph
G = nx.Graph()

# Add rooms as nodes
for room in project['rooms']:
    G.add_node(room['id'], name=room['name'])

# Add doors as edges
for door in project['doors']:
    if door.get('room_from') and door.get('room_to'):
        G.add_edge(
            door['room_from'],
            door['room_to'],
            door_id=door['id'],
            width=door['width'],
            is_egress=door['is_egress']
        )

print(f"📊 Graph Statistics:")
print(f"   Nodes (rooms): {G.number_of_nodes()}")
print(f"   Edges (doors): {G.number_of_edges()}")
print(f"   Connected: {nx.is_connected(G)}")


In [ ]:
# Find evacuation routes
egress_rooms = []
for door in project['doors']:
    if door['is_egress']:
        if door.get('room_from'):
            egress_rooms.append(door['room_from'])
        if door.get('room_to'):
            egress_rooms.append(door['room_to'])

print("🚨 Evacuation Analysis\n")

for room in project['rooms']:
    room_id = room['id']
    
    # Find shortest path to exit
    if egress_rooms:
        try:
            path = nx.shortest_path(G, room_id, egress_rooms[0])
            distance = len(path) - 1  # Number of doors to cross
            
            path_names = [G.nodes[r]['name'] for r in path]
            
            print(f"🏢 {room['name']}")
            print(f"   Path: {' → '.join(path_names)}")
            print(f"   Doors to cross: {distance}")
            print()
        except:
            print(f"🏢 {room['name']}")
            print(f"   ❌ No path to exit!")
            print()


## 🎯 Summary

In this tutorial, you learned:

1. ✅ Calculate room areas with Shapely
2. ✅ Build circulation graphs with NetworkX
3. ✅ Find shortest evacuation paths
4. ✅ Visualize building connectivity

**Next**: Tutorial 3 - RAG (Query Building Codes)
